[eosio.token.cpp](https://github.com/EOSIO/eosio.contracts/blob/c046863a65d7e98424312ee8009f0acb493e6231/contracts/eosio.token/src/eosio.token.cpp#L87)

```C++
void token::transfer( name    from,
                      name    to,
                      asset   quantity,
                      string  memo )
{
    check( from != to, "cannot transfer to self" );
    require_auth( from );
    check( is_account( to ), "to account does not exist");
    auto sym = quantity.symbol.code();
    stats statstable( _self, sym.raw() );
    const auto& st = statstable.get( sym.raw() );

    require_recipient( from );
    require_recipient( to );

    check( quantity.is_valid(), "invalid quantity" );
    check( quantity.amount > 0, "must transfer positive quantity" );
    check( quantity.symbol == st.supply.symbol, "symbol precision mismatch" );
    check( memo.size() <= 256, "memo has more than 256 bytes" );

    auto payer = has_auth( to ) ? to : from;

    sub_balance( from, quantity );
    add_balance( to, quantity, payer );
}
```

In [1]:
from init import *

In [5]:
code = r'''
#include <string>
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/action.hpp>
#include <eosio/asset.hpp>

using namespace std;
using namespace eosio;

struct transfer {
    name from;
    name to;
    asset amount;
    string memo;
    EOSLIB_SERIALIZE( transfer, (from)(to)(amount)(memo) )
};

extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        print(name(receiver), " ", name(code), " ", name(action), "\n");
        if (name(code) == "eosio.token"_n && name(action) == "transfer"_n) {
            auto t = unpack_action_data<transfer>();
            if (t.to == "eosio"_n && t.amount.symbol == symbol("TNT", 4)) {
                print(t.memo, "\n");
            }
        }
    }
}
'''

code = uuosapi.compile(test_account1, code, vm_type=0)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [6]:
r = uuosapi.transfer(test_account1, 'eosio', 0.1, 'hello,world')
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['action_traces'][0]['inline_traces'][0]['console'])
print(r['processed']['action_traces'][0]['inline_traces'][1]['console'])
print(r['processed']['elapsed'])



wkpmdjdsztyu eosio.token transfer
hello,world


1277
